# Read in Data

In [24]:
# Import data.
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)

[nltk_data] Downloading package punkt to /Users/judychen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/judychen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: exp

In [25]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [26]:
# Check missing values.
df.isnull().sum()

marketplace            0
customer_id            0
review_id              0
product_id             0
product_parent         0
product_title          2
product_category       0
star_rating            0
helpful_votes          0
total_votes            0
vine                   0
verified_purchase      0
review_headline        7
review_body          148
review_date            4
dtype: int64

In [27]:
# Remove if the review without review boday.
df.dropna(subset=['review_body'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [28]:
# Use the first 1000 data as training data.
data = df.loc[:999, 'review_body'].tolist()

# Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.

In [29]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print("Stopwords that we use from nltk library: ")
print(stopwords)

Stopwords that we use from nltk library: 
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

In [30]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# Define functions to tokenize and stem reviews.
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string
    tokens = []
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())
    
    # filter out any tokens not containing letters such as numeric tokens and raw punctuation.
    filtered_tokens = []
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
    
    # stemming.
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# Test function with the first review.
print(data[0])
print(tokenization_and_stemming(data[0]))


Absolutely love this watch! Get compliments almost every time I wear it. Dainty.
['absolut', 'love', 'get', 'compliment', 'almost', 'everi', 'time', 'wear', 'dainti']


# Term Frequency - Inverse Document Frequency

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Define vectorizer parameters, and use TfidfVectorizer to create tf-idf matrix
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000, min_df=0.01, stop_words='english', use_idf=True, 
                                tokenizer=tokenization_and_stemming, ngram_range=(1,1))

# Fit the vectorizer to synopses
tfidf_matrix = tfidf_model.fit_transform(data)
tfidf_matrix

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6891 stored elements in Compressed Sparse Row format>

In [32]:
tfidf_matrix.toarray()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [33]:
tfidf_matrix.todense()

matrix([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [34]:
# Save terms identified by TF-IDF
tf_selected_words = tfidf_model.get_feature_names()
print(tf_selected_words)

['abl', 'absolut', 'accur', 'actual', 'adjust', 'alarm', 'alreadi', 'alway', 'amaz', 'amazon', 'anoth', 'arm', 'arriv', 'automat', 'awesom', 'bad', 'band', 'batteri', 'beauti', 'best', 'better', 'big', 'bit', 'black', 'blue', 'bought', 'box', 'bracelet', 'brand', 'break', 'bright', 'broke', 'button', 'buy', 'ca', 'came', 'case', 'casio', 'chang', 'cheap', 'clasp', 'classi', 'clock', 'color', 'come', 'comfort', 'compliment', 'cool', 'cost', 'crown', 'crystal', 'dark', 'date', 'daughter', 'day', 'deal', 'definit', 'deliveri', 'design', 'dial', 'differ', 'difficult', 'disappoint', 'display', 'dress', 'durabl', 'easi', 'easili', 'end', 'everi', 'everyday', 'everyth', 'exact', 'excel', 'expect', 'expens', 'face', 'fair', 'far', 'fast', 'featur', 'feel', 'fell', 'fine', 'finish', 'fit', 'function', 'gave', 'gift', 'gold', 'good', 'got', 'great', 'hand', 'happi', 'hard', 'heavi', 'high', 'hold', 'honest', 'hope', 'hour', 'howev', 'husband', 'includ', 'instruct', 'invicta', 'issu', 'item', 'ke

# K-means Clustering

In [35]:
# k-means clustering
from sklearn.cluster import KMeans

# define number of clusters
num_clusters = 10
km = KMeans(n_clusters= num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [36]:
# Analyze K-means Result
# Create dataframe films from all of the input files.
product = {'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,8
1,I love this watch it keeps time wonderfully.,8
2,Scratches,2
3,"It works well on me. However, I found cheaper ...",2
4,Beautiful watch face. The band looks nice all...,0
5,"i love this watch for my purpose, about the pe...",8
6,"for my wife and she loved it, looks great and ...",1
7,I was about to buy this thinking it was a Swis...,2
8,Watch is perfect. Rugged with the metal &#34;B...,1
9,Great quality and build.<br />The motors are r...,2


In [37]:
# Number of reviews included in each cluster.
frame['cluster'].value_counts().to_frame()

,cluster
2,429
8,97
0,91
7,77
1,70
5,64
3,63
6,55
4,30
9,24


In [38]:
# km.cluster_centers_ denotes the importances of each items in centroid
km.cluster_centers_

array([[0.00180128, 0.00225074, 0.01222436, ..., 0.0023171 , 0.04946018,
        0.02021277],
       [0.00460702, 0.        , 0.        , ..., 0.        , 0.        ,
        0.01694012],
       [0.00775552, 0.00317823, 0.00256665, ..., 0.00785089, 0.01175987,
        0.01899595],
       ...,
       [0.00365974, 0.02944326, 0.        , ..., 0.00652463, 0.02104069,
        0.        ],
       [0.        , 0.03238075, 0.        , ..., 0.01398553, 0.01444061,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02074231, 0.        ,
        0.        ]])

In [39]:
# Clustering result by K-means
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()

    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")

Cluster 0 words:band,look,leather,wrist,good,nice,
Cluster 0 reviews (91 reviews): 
Cluster 1 words:great,look,price,comfort,product,work,
Cluster 1 reviews (70 reviews): 
Cluster 2 words:work,look,time,use,day,good,
Cluster 2 reviews (429 reviews): 
Cluster 3 words:good,product,price,love,big,qualiti,
Cluster 3 reviews (63 reviews): 
Cluster 4 words:excel,product,price,fast,qualiti,good,
Cluster 4 reviews (30 reviews): 
Cluster 5 words:like,look,work,face,nice,big,
Cluster 5 reviews (64 reviews): 
Cluster 6 words:nice,price,look,realli,good,love,
Cluster 6 reviews (55 reviews): 
Cluster 7 words:perfect,beauti,fit,wear,great,realli,
Cluster 7 reviews (77 reviews): 
Cluster 8 words:love,wife,husband,look,color,gift,
Cluster 8 reviews (97 reviews): 
Cluster 9 words:expect,clock,everyth,thank,nice,time,
Cluster 9 reviews (24 reviews): 


# Topic Modeling - Latent Dirichlet Allocation

In [40]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10)

In [41]:
# document topic matrix for tfida_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output)

[[0.02961289 0.02961349 0.0296127  ... 0.02961471 0.41336188 0.02961335]
 [0.04146564 0.04146658 0.04146146 ... 0.04147284 0.04148658 0.0414698 ]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 ...
 [0.05000009 0.05       0.05       ... 0.05       0.05       0.05000115]
 [0.03336732 0.03336634 0.03337834 ... 0.03336842 0.03336949 0.03337937]
 [0.03361466 0.03361466 0.0336204  ... 0.03362321 0.17279604 0.55823218]]


In [42]:
# Topics and words matrix
topic_word = lda.components_
print(topic_word)

[[0.1        0.10000043 0.10006528 ... 0.10001659 0.10001513 5.37321507]
 [0.10000707 0.10001138 0.1        ... 0.22495653 0.10000407 0.10000758]
 [0.10000079 0.10000545 0.1000089  ... 5.91058679 0.10000177 0.57897091]
 ...
 [0.10000453 0.10001555 0.10000129 ... 0.10000387 2.87478919 0.10000443]
 [0.10001472 7.51748258 0.10000002 ... 0.10000458 0.10000136 0.10000565]
 [1.02152978 0.10007015 2.65366751 ... 0.10004474 3.91535088 3.35201994]]


In [43]:
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]
doc_names = ['Doc' + str(i) for i in range(len(data))]
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic
df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,topic
Doc0,0.03,0.03,0.03,0.03,0.03,0.03,0.35,0.03,0.41,0.03,8
Doc1,0.04,0.04,0.04,0.63,0.04,0.04,0.04,0.04,0.04,0.04,3
Doc2,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc3,0.03,0.03,0.03,0.03,0.74,0.03,0.03,0.03,0.03,0.03,4
Doc4,0.47,0.02,0.02,0.02,0.38,0.02,0.02,0.02,0.02,0.02,0
Doc5,0.04,0.04,0.67,0.04,0.04,0.04,0.04,0.04,0.04,0.04,2
Doc6,0.03,0.03,0.35,0.03,0.03,0.39,0.03,0.03,0.03,0.03,5
Doc7,0.03,0.03,0.03,0.03,0.72,0.03,0.03,0.03,0.03,0.03,4
Doc8,0.02,0.02,0.02,0.02,0.55,0.02,0.02,0.27,0.02,0.02,4
Doc9,0.03,0.03,0.03,0.37,0.03,0.03,0.03,0.03,0.03,0.41,9


In [44]:
df_document_topic['topic'].value_counts().to_frame()

,topic
4,209
9,182
5,112
3,105
0,101
1,86
7,65
2,56
8,54
6,30


In [45]:
# topic word matrix
print(lda.components_)
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names
df_topic_words.head()

[[0.1        0.10000043 0.10006528 ... 0.10001659 0.10001513 5.37321507]
 [0.10000707 0.10001138 0.1        ... 0.22495653 0.10000407 0.10000758]
 [0.10000079 0.10000545 0.1000089  ... 5.91058679 0.10000177 0.57897091]
 ...
 [0.10000453 0.10001555 0.10000129 ... 0.10000387 2.87478919 0.10000443]
 [0.10001472 7.51748258 0.10000002 ... 0.10000458 0.10000136 0.10000565]
 [1.02152978 0.10007015 2.65366751 ... 0.10004474 3.91535088 3.35201994]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,0.100000,0.100000,0.100065,0.100000,0.100011,2.279286,0.100005,0.100008,0.100008,0.568677,...,0.100000,0.100120,0.100000,0.100000,0.100000,3.042194,0.100004,0.100017,0.100015,5.373215
Topic1,0.100007,0.100011,0.100000,0.100013,0.100019,0.100000,0.100003,0.100001,7.739703,0.100035,...,0.100002,0.100000,0.100003,0.100000,2.639144,1.897503,0.100000,0.224957,0.100004,0.100008
Topic2,0.100001,0.100005,0.100009,0.100006,0.100056,0.100000,3.445684,0.100008,0.100000,1.242009,...,0.100003,0.100042,11.742352,2.291155,0.100002,0.100003,0.100000,5.910587,0.100002,0.578971
Topic3,0.100003,0.100001,0.100000,0.100004,0.100000,0.100000,0.100001,2.192505,0.100002,0.100000,...,0.100009,0.100000,0.100002,0.100000,0.100003,0.100008,0.100000,0.100003,0.100001,0.100004
Topic4,3.273765,0.178173,0.100012,1.990626,0.475633,1.389875,0.100015,1.432605,0.940885,4.291749,...,0.100028,0.100053,0.100013,0.100016,0.100070,4.565381,3.303069,0.100051,6.923597,2.692868


In [46]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,beauti,broke,batteri,week,love,arriv,color,band,year,water,clock,resist,fell,day,anoth
Topic 1,nice,awesom,thank,amaz,super,fast,price,ship,simpl,set,love,look,great,instruct,hard
Topic 2,perfect,wife,gift,love,worth,recommend,happi,quick,buy,alreadi,movement,gave,look,high,nice
Topic 3,good,love,like,qualiti,realli,pretti,price,better,person,look,pictur,beauti,time,month,came
Topic 4,look,band,face,like,great,pictur,nice,watch,cheap,leather,feel,make,time,design,size
Topic 5,great,excel,work,product,expect,price,look,seller,exact,item,everyth,perfect,fit,valu,good
Topic 6,strap,compliment,invicta,lot,buy,dress,easili,everi,style,think,love,sever,short,anoth,watch
Topic 7,husband,love,deliveri,pleas,big,stylish,heavi,receiv,got,daughter,look,piec,surpris,beauti,time
Topic 8,absolut,wear,cool,bought,want,love,bad,fine,money,time,work,reason,swim,tell,beauti
Topic 9,use,band,look,work,comfort,day,great,time,far,easi,good,light,function,deal,like
